SDAT information is maintained from a baseline that is updated to Q2 2020, available in a zip file on this link.  The file interest is: Dorc2017.dbf  
https://www.dropbox.com/s/oc1l1frorg66vlr/DORC_MPV17.zip
extract it using: $> tar -xf DORC_MPV17.zip Dorc2017/ATDATA/DATABASE/Dorc2017dbf

Updating this file requires downloading monthly (or quarterly) changes listed on this page:
https://planning.maryland.gov/Pages/OurProducts/DownloadFiles.aspx

All of the updates through 4/30/2021 have been aggegated in a single file written from this notebook.

In [2]:
#!pip install simpledbf
import pandas as pd
from simpledbf import Dbf5

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dbf = Dbf5('drive/My Drive/SDAT/Dorc2017.dbf')
df = dbf.to_dataframe()

In [5]:
df = df.set_index('acctid')
df

,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,dr1folio,...,assessor,transno1,grntnam1,gr1clrk1,gr1libr1,gr1folo1,convey1,tradate,considr1,mortgag1,nfmlndvl,nfmimpvl,nfmttlvl,crtarcod,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate
acctid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000012,DORC,508948.5,110654.0,24019970100,240199701001,80,N,SF,5727 ADAMS ROAD,5727,NaN,ADAMS,RD,NaN,NaN,P,FEDERALSBURG,21632,NAGEL RICHARD LEE & CONNIE JANE,NaN,NAGEL RICHARD LEE & CONNI,5714 ADAMS RD,NaN,FEDERALSBURG,MD,21632,1700,5727,NaN,ADAMS,RD,FEDERALSBURG,21632,NaN,52.94 ACRES,S/S ADAMS RD.,NE OF FINCHVILLE,MLB,00363,0779,...,1014,980275,"ADAMS, MAGGIE M.",NaN,NaN,NaN,4,19971009,81200,0,22100,0,22100,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,2,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN
1001000020,DORC,505757.4,99135.2,24019970100,240199701002,80,D,SF,5430 INDIANTOWN ROAD,5430,NaN,INDIANTOWN,RD,NaN,NaN,P,RHODESDALE,21659,PLEASANTS THOMAS L,PLEASANTS ARLENE F,PLEASANTS THOMAS L,5430 INDIANTOWN RD,NaN,RHODESDALE,MD,21659,NaN,5430,NaN,INDIANTOWN,RD,RHODESDALE,21659,NaN,IMPS58.963 ACRES,W/S INDIANTOWN RD.,S OF BROOKVIEW,PLC,00266,0744,...,1014,910736,"ALLEN, CHARLES WALTER AND",NaN,NaN,NaN,1,19910314,160000,0,71800,266300,338100,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,1,2,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN
1001000039,DORC,511216.1,106713.9,24019970100,240199701002,80,H,SF,6009 COKESBURY ROAD,6009,NaN,COKESBURY,RD,NaN,NaN,P,RHODESDALE,19973,GARDINER KEVIN E,GARDINER LORI A,GARDINER KEVIN E,6009 COKESBURY RD,NaN,SEAFORD,DE,19973,NaN,6009,NaN,COKESBURY,RD,SEAFORD,19973,NaN,IMPS4.80 ACRES,E/S COKESBURY RD,SW/RELIANCE,NaN,01493,0455,...,1014,NaN,MELENDEZ EDWIN,NaN,01403,0178,1,20181115,310000,0,39600,247500,287100,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,1,0,1,0,0,0,0,0,1,2,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN
1001000047,DORC,508807.3,110360.1,24019970100,240199701001,80,N,SF,5731 DAVIS MILL POND ROAD,5731,NaN,DAVIS MILL POND,RD,NaN,NaN,P,FEDERALSBURG,21632,HARIM MILLSBORO LLC,NaN,HARIM MILLSBORO LLC,PO BOX 1380,MAILSTOP 100484,MILLSBORO,DE,19966,NaN,5731,NaN,DAVIS MILL POND,RD,NaN,NaN,NaN,IMPS20 ACRES,W/S DAVIS MILLPOND RD,NE/FINCHVILLE,NaN,01471,0011,...,1014,NaN,ALLEN HARIM FARMS LLC,NaN,01067,0259,4,20180628,10,0,33000,57400,90400,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,1,2,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN
1001000055,DORC,507295.0,112993.8,24019970100,240199701001,80,N,TR,6940 RELIANCE ROAD,6940,NaN,RELIANCE,RD,NaN,NaN,P,FEDERALSBURG,21632,HARIM MILLSBORO LLC,NaN,HARIM MILLSBORO LLC,PO BOX 1380,MAILSTOP 100484,MILLSBORO,DE,19966,NaN,6940,NaN,RELIANCE,RD,NaN,NaN,NaN,IMPS232 AC,S/W ALLENS COR-FDG RD,W/ALLENS COR,NaN,01471,0011,...,1014,NaN,ALLEN HARIM FARMS LLC,NaN,01067,0259,4,20180628,10,0,61000,120000,181000,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,1,0,0,2,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007162065,DORC,483386.9,98718.8,24019970702,240199707021,81,N,S

In [6]:
from os import walk
print(len(df))
for (dirpath, dirnames, filenames) in walk('drive/My Drive/SDAT/'):
    for file in ([name for name in sorted(filenames) if 'SALE' in name.upper()]):
      print(dirpath+file)
      add_df = Dbf5(dirpath+file)
      new_df = add_df.to_dataframe()
      new_df.columns = [col.lower() for col in new_df.columns]
      new_df = new_df.query('jurscode == "DORC"').set_index('acctid')
      new_df = new_df[~new_df.index.duplicated(keep='last')]
      updates = [str(v) for v in set(df.index.values).intersection(set(new_df.index.values)) if int(v) > 0]
      df.loc[updates] = new_df.loc[updates]
      appends = list(set(new_df.index.values).difference(set(updates)))
      df = df.append(new_df.loc[appends])
    break
df.drop_duplicates(inplace=True)
len(df)

23202
drive/My Drive/SDAT/SALE0420.dbf
drive/My Drive/SDAT/SALE0520.dbf
drive/My Drive/SDAT/SALE0620.dbf
drive/My Drive/SDAT/SALE0720.dbf
drive/My Drive/SDAT/SALE0820.dbf
drive/My Drive/SDAT/SALE1020.dbf
drive/My Drive/SDAT/SALE1120.dbf
drive/My Drive/SDAT/SALE1220.dbf
drive/My Drive/SDAT/Sale0121.dbf
drive/My Drive/SDAT/Sale0221.dbf
drive/My Drive/SDAT/Sale0321.dbf
drive/My Drive/SDAT/Sale0421.dbf


23214

In [7]:
can_df = pd.read_csv("drive/My Drive/pita 2020/20191225-can_rental_study_updated.csv")[['ACCTID','CAN_GROUP','CAN_OWNCLASS','GEOLATLON','GEOHASH']]
can_df.ACCTID = can_df.ACCTID.apply(lambda x: str(x).strip())
can_df.rename(columns={'ACCTID':'acctid'},inplace=True)
can_df.drop_duplicates(inplace=True)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,9,10,28,43,85,109,110) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
can_df.set_index('acctid').to_csv('drive/My Drive/SDAT/CAN-ref.csv')

In [9]:
geoinfo = pd.read_csv("drive/My Drive/SDAT/CAN-ref.csv").set_index('acctid')
df.join(geoinfo,how='inner')

,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,dr1folio,...,nfmimpvl,nfmttlvl,crtarcod,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,saleid,curlndvl,curimpvl,curttlvl,sallndvl,salimpvl,salttlvl,CAN_GROUP,CAN_OWNCLASS,GEOLATLON,GEOHASH
acctid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1007104014,DORC,480297.2,99307.0,24019970600,240199706001,81,N,SF,405 E APPLEBY AVE,405,E,APPLEBY,AVE,NaN,NaN,P,CAMBRIDGE,21613,INSLEY BARBARA D,NaN,INSLEY BARBARA D,105 MAKO DR,NaN,CAMBRIDGE,MD,21613,3019,405,NaN,E APPLEBY,AVE,CAMBRIDGE,21613,NaN,"IMPSLOT 17-8,008 SQ. FT.",405 APPLEBY AVENUE,CAMBRIDGE,MLB,00374,0428,...,63200.0,84600.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,A,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOUSING,"38.55764256253734, -76.07872406520042",dqcgkg1nt1x8
1007104022,DORC,480574.7,99966.0,24019970600,240199706001,81,H,SF,717 PEACHBLOSSOM AVE,717,NaN,PEACHBLOSSOM,AVE,NaN,NaN,P,CAMBRIDGE,21613,AARON GEORGE ROBERT & PEGGY D,L/E THEN TO ROBIN L AARON ETAL,AARON GEORGE ROBERT & PEG,717 PEACHBLOSSOM AVENUE,NaN,CAMBRIDGE,MD,21613,2354,717,NaN,PEACHBLOSSOM,AVE,CAMBRIDGE,21613,NaN,"IMPSLOT 4,000 SQ. FT.",717 PEACHBLOSSOM AVENUE,CAMBRIDGE,MLB,00464,0616,...,54800.0,84900.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,A,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOUSING,"38.56355375091615, -76.07546386301141",dqcgku78cpqw
1007104057,DORC,481347.2,100263.6,24019970400,240199704002,81,N,SF,304 DORCHESTER AVE,304,NaN,DORCHESTER,AVE,NaN,NaN,P,CAMBRIDGE,21613,FITZHUGH WILLIAM R &,FITZHUGH WILLIAM R JR,FITZHUGH WILLIAM R &,5522 WHITEHALL RD,NaN,CAMBRIDGE,MD,21613,3442,304,NaN,DORCHESTER,AVE,CAMBRIDGE,21613,NaN,IMPSLT 22 AND P/O #21,"14,700 SQ. FT.",304 DORCHESTER AVE,PLC,00300,0622,...,61800.0,93800.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOUSING,"38.56616379596679, -76.06656568677926",dqcgmhf00w18
1007104065,DORC,480328.3,99420.7,24019970600,240199706001,81,H,NaN,413 SHEPHERD AVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAMBRIDGE,21613,GENOVESE LISA M,NaN,GENOVESE LISA M,413 SHEPHERD AVE,NaN,CAMBRIDGE,MD,21613,NaN,413,NaN,SHEPHERD,AVE,CAMBRIDGE,21613,NaN,"IMPVLOT 14,400 SQ. FT.",413 SHEPHERD AVENUE,CAMBRIDGE,NaN,01633,0271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,https://sdat.dat.maryland.gov/RealProperty/Pag...,MDPV2017_18,NaN,NaN,22000.0,85500.0,0.0,22000.0,85500.0,0.0,NaN,HOUSING,"38.55866397721814, -76.07835409909252",dqcgkg3kmzgt
1007104073,DORC,480302.8,99425.4,24019970600,240199706001,81,N,SF,415 SHEPHERD AVE,415,NaN,SHEPHERD,AVE,NaN,NaN,P,CAMBRIDGE,21613,RUDDY DUCK LLC,NaN,RUDDY DUCK LLC,PO BOX 1181,NaN,CAMBRIDGE,MD,21613,NaN,415,NaN,SHEPHERD,AVE,CAMBRIDGE,21613,NaN,"IMPSLOT 84&85-15,696 SQ.",415 SHEPHERD AVENUE,CAMBRIDGE,NaN,01335,0238,...,115500.0,137600.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OTTER_RUDDY_GADWALL,HOUSING,"38.5587086350541, -76.07864610896351",dqcgkg3hz03b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [11]:
new_df.query('acctid == "1001000020"')

,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,address,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,dr1folio,towncode,desctown,subdivsn,dsubcode,descsubd,plat,section,block,...,depth,pfuw,pfus,pflw,pfsp,pfsu,pfic,pfih,recind,yearblt,sqftstrc,strugrad,descgrad,strucnst,desccnst,strustyl,descstyl,strubldg,descbldg,lastinsp,lastassd,assessor,transno1,grntnam1,gr1clrk1,gr1libr1,gr1folo1,convey1,tradate,considr1,mortgag1,curlndvl,curimpvl,curttlvl,sallndvl,salimpvl,salttlvl,ptype,sdatwebadr,existing
acctid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000020,DORC,505757.4,99135.2,24019970100,240199701002,80,D,5430 INDIANTOWN ROAD,RHODESDALE,21659,PLEASANTS THOMAS R,NaN,PLEASANTS THOMAS R,5430 INDIANTOWN RD,NaN,RHODESDALE,MD,21659,NaN,5430,NaN,INDIANTOWN,RD,RHODESDALE,21659,NaN,IMPV58.963 ACRES,W/S INDIANTOWN RD.,S OF BROOKVIEW,NaN,01595,0347,NaN,NaN,0000,010000,N/A,NaN,NaN,NaN,...,0.0,1,2,0,0,0,0,0,0,1976,3028,4,Codes range from lowest to highest quality 1-9,001,CNST Siding,001,STRY 1 Story No Basement,001,DWEL Standard Unit,201709,189912,1014,NaN,PLEASANTS THOMAS L & ARLENE F,PLC,00266,0744,4,20200914,0,0,71800,266300,0,71800,266300,0,2,https://sdat.dat.maryland.gov/RealProperty/Pag...,MDPV2017_18


In [12]:
df.query('acctid == "1001000020"')

,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,dr1folio,...,tradate,considr1,mortgag1,nfmlndvl,nfmimpvl,nfmttlvl,crtarcod,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,saleid,curlndvl,curimpvl,curttlvl,sallndvl,salimpvl,salttlvl
acctid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000020,DORC,505757.4,99135.2,24019970100,240199701002,80,D,NaN,5430 INDIANTOWN ROAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RHODESDALE,21659,PLEASANTS THOMAS R,NaN,PLEASANTS THOMAS R,5430 INDIANTOWN RD,NaN,RHODESDALE,MD,21659,NaN,5430,NaN,INDIANTOWN,RD,RHODESDALE,21659,NaN,IMPV58.963 ACRES,W/S INDIANTOWN RD.,S OF BROOKVIEW,NaN,01595,0347,...,20200914,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,https://sdat.dat.maryland.gov/RealProperty/Pag...,MDPV2017_18,NaN,NaN,71800.0,266300.0,0.0,71800.0,266300.0,0.0


In [13]:
assessments = pd.read_csv('https://opendata.maryland.gov/resource/ed4q-f8tm.csv?jurisdiction_code_mdp_field_jurscode=DORC&$limit=25000')
#assessments = pd.read_csv("https://opendata.maryland.gov/resource/ye3m-tr66.csv?$limit=25000")
assessment_fields = ['account_id_mdp_field_acctid','real_property_search_link',
                     'search_google_maps_for_this_location',
                     'c_a_m_a_system_data_structure_area_sq_ft_mdp_field_sqftstrc_sdat_field_241',
                     'current_assessment_year_total_phase_in_value_sdat_field_171',
                     'c_a_m_a_system_data_year_built_yyyy_mdp_field_yearblt_sdat_field_235',
                     'premise_address_number_mdp_field_premsnum_sdat_field_20',
                     'premise_address_number_suffix_sdat_field_21',
                     'premise_address_direction_mdp_field_premsdir_sdat_field_22',
                     'premise_address_name_mdp_field_premsnam_sdat_field_23',
                     'premise_address_type_mdp_field_premstyp_sdat_field_24',
                     'premise_address_city_mdp_field_premcity_sdat_field_25',
                     'premise_address_zip_code_mdp_field_premzip_sdat_field_26',
                     'mdp_street_address_mdp_field_address']
assessments = assessments[assessment_fields]
#assessment_column_names = ['acctid','sdat','google_maps','struct_sqft','assessed_value','yearblt','address_number','address_unit_id','street_direction','street_name','street_type','city','zipcode','address']
assessment_column_names = ['acctid','sdat','google_maps','struct_sqft','assessed_value','address_number','address_unit_id','street_direction','street_name','street_type']
assessments.rename(columns=dict(zip(assessment_fields,assessment_column_names)),inplace=True)
assessments.acctid = assessments.acctid.astype(str)
assessments.set_index('acctid',inplace=True)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (25,31,38,39,41,42,63,166,172,211,212) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
df = df.join(assessments,how='inner')

In [15]:
df.to_csv('drive/My Drive/pita 2021/SDAT-CAN-ref-202104.csv')

In [16]:
df

,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,dr1folio,...,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,saleid,curlndvl,curimpvl,curttlvl,sallndvl,salimpvl,salttlvl,sdat,google_maps,struct_sqft,assessed_value,address_number,address_unit_id,street_direction,street_name,street_type,premise_address_type_mdp_field_premstyp_sdat_field_24,premise_address_city_mdp_field_premcity_sdat_field_25,premise_address_zip_code_mdp_field_premzip_sdat_field_26,mdp_street_address_mdp_field_address
acctid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000012,DORC,508948.5,110654.0,24019970100,240199701001,80,N,SF,5727 ADAMS ROAD,5727,NaN,ADAMS,RD,NaN,NaN,P,FEDERALSBURG,21632,NAGEL RICHARD LEE & CONNIE JANE,NaN,NAGEL RICHARD LEE & CONNI,5714 ADAMS RD,NaN,FEDERALSBURG,MD,21632,1700,5727,NaN,ADAMS,RD,FEDERALSBURG,21632,NaN,52.94 ACRES,S/S ADAMS RD.,NE OF FINCHVILLE,MLB,00363,0779,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.65678537...,0,22100,0,5727,NaN,NaN,ADAMS,RD,FEDERALSBURG,21632.0,5727 ADAMS ROAD
1001000020,DORC,505757.4,99135.2,24019970100,240199701002,80,D,NaN,5430 INDIANTOWN ROAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RHODESDALE,21659,PLEASANTS THOMAS R,NaN,PLEASANTS THOMAS R,5430 INDIANTOWN RD,NaN,RHODESDALE,MD,21659,NaN,5430,NaN,INDIANTOWN,RD,RHODESDALE,21659,NaN,IMPV58.963 ACRES,W/S INDIANTOWN RD.,S OF BROOKVIEW,NaN,01595,0347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,https://sdat.dat.maryland.gov/RealProperty/Pag...,MDPV2017_18,NaN,NaN,71800.0,266300.0,0.0,71800.0,266300.0,0.0,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.55341338...,3028,338100,1976,5430,NaN,NaN,INDIANTOWN,RD,RHODESDALE,21659.0,5430 INDIANTOWN ROAD
1001000039,DORC,511216.1,106713.9,24019970100,240199701002,80,H,SF,6009 COKESBURY ROAD,6009,NaN,COKESBURY,RD,NaN,NaN,P,RHODESDALE,19973,GARDINER KEVIN E,GARDINER LORI A,GARDINER KEVIN E,6009 COKESBURY RD,NaN,SEAFORD,DE,19973,NaN,6009,NaN,COKESBURY,RD,SEAFORD,19973,NaN,IMPS4.80 ACRES,E/S COKESBURY RD,SW/RELIANCE,NaN,01493,0455,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.62101071...,3492,287100,2004,6009,NaN,NaN,COKESBURY,RD,SEAFORD,19973.0,6009 COKESBURY ROAD
1001000047,DORC,508807.3,110360.1,24019970100,240199701001,80,N,SF,5731 DAVIS MILL POND ROAD,5731,NaN,DAVIS MILL POND,RD,NaN,NaN,P,FEDERALSBURG,21632,HARIM MILLSBORO LLC,NaN,HARIM MILLSBORO LLC,PO BOX 1380,MAILSTOP 100484,MILLSBORO,DE,19966,NaN,5731,NaN,DAVIS MILL POND,RD,NaN,NaN,NaN,IMPS20 ACRES,W/S DAVIS MILLPOND RD,NE/FINCHVILLE,NaN,01471,0011,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.65415540...,1438,90400,1920,5731,NaN,NaN,DAVIS MILL POND,RD,NaN,0.0,5731 DAVIS MILL POND ROAD
1001000055,DORC,507295.0,112993.8,24019970100,240199701001,80,N,TR,6940 RELIANCE ROAD,6940,NaN,RELIANCE,RD,NaN,NaN,P,FEDERALSBURG,21632,HARIM MILLSBORO LLC,NaN,HARIM MILLSBORO LLC,PO BOX 1380,MAILSTOP 100484,MILLSBORO,DE,19966,NaN,6940,NaN,RELIANCE,RD,NaN,NaN,NaN,IMPS23